### What this script does

- Loads corrected LWP (MWR) and combined SWR/LWR parquet files.

- Computes solar position and clear-sky GHI (pvlib/Ineichen), merges with data.

- Builds plots of SW↓ vs apparent zenith, colors by LWP, plus binned summaries.

- Computes CSI and filters by LWP < 300 g/m².

- (Optional) Loads Cabauw dataset, adds solar geometry/clear-sky GHI, and compares Warmenhuizen vs Cabauw curves by LWP bins.

#### Edit before running
1) Your local data folders:
- BASE_DIR_MWR = r"C:\path\to\your\Microwave_radiometer"
- BASE_FOLDER  = r"C:\path\to\your\radiometer"

2) Parquet filenames (change only if yours differ)
- FILENAME_LWP_CORRECTED = "Corrected_LWP_Data.parquet"
- FILENAME_SWR = "Combined_SWR_Data.parquet"
- FILENAME_LWR = "Combined_LWR_Data.parquet"    # (loaded if you decide to use it)

3) Site coordinates (used by pvlib)
- LATITUDE  = 52.6324
- LONGITUDE = 4.7534
- ALTITUDE  = 0

4) Optional: Cabauw dataset path (enable only if you have this file):

   cabauw_parquet_path = r"C:\path\to\your\Cabauw\cabauw_merged_data_April_June_July.parquet"

5) Plot/filters you may want to tweak

In [ ]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pvlib
import matplotlib as mpl


In [ ]:

## Edit before running!!!
# 1) Your local data folders:
# Example:
# BASE_DIR_MWR = r"D:\Thesis\data\Microwave_radiometer"
# BASE_FOLDER  = r"D:\Thesis\data\radiometer"
BASE_DIR_MWR = r"C:\path\to\your\Microwave_radiometer"
BASE_FOLDER  = r"C:\path\to\your\radiometer"


FILENAME_LWP_CORRECTED = 'Corrected_LWP_Data.parquet'
FILENAME_SWR = 'Combined_SWR_Data.parquet'
FILENAME_LWR = 'Combined_LWR_Data.parquet'
# Constants
LATITUDE = 52.6324  # Alkmaar, Netherlands
LONGITUDE = 4.7534
ALTITUDE = 0


In [ ]:
# Function to load a Parquet file
def load_parquet_file(base_dir, filename):
    file_path = os.path.join(base_dir, filename)
    return pd.read_parquet(file_path)

def calculate_solar_position_and_clear_sky(latitude, longitude, timestamps):
    # Calculate solar position
    solar_position = pvlib.solarposition.get_solarposition(timestamps, latitude, longitude)

    # Create location object
    location = pvlib.location.Location(latitude, longitude)

    # Calculate clear-sky GHI using the Ineichen model
    clearsky = location.get_clearsky(timestamps, model='ineichen')
    
    return solar_position, clearsky['ghi']


In [ ]:
def load_and_prepare_data():
    """Load and prepare the initial merged data."""
    # Load LWP corrected data
    df_lwp_corrected = load_parquet_file(BASE_DIR_MWR, FILENAME_LWP_CORRECTED)
    df_combined_swr = load_parquet_file(BASE_FOLDER, FILENAME_SWR)

    # Ensure TIMESTAMP columns are in datetime format
    df_lwp_corrected['TIMESTAMP'] = pd.to_datetime(df_lwp_corrected['TIMESTAMP'], errors='coerce')
    df_combined_swr['TIMESTAMP'] = pd.to_datetime(df_combined_swr['TIMESTAMP'], errors='coerce')
    
    # Drop rows with NaT in TIMESTAMP
    df_lwp_corrected.dropna(subset=['TIMESTAMP'], inplace=True)
    df_combined_swr.dropna(subset=['TIMESTAMP'], inplace=True)
    
    # Filter SWR data
    df_combined_swr = df_combined_swr[df_combined_swr['SR15D1Dn_Irr'] > 10].reset_index(drop=True)
    
   
    #df_combined_swr['TIMESTAMP'] = df_combined_swr['TIMESTAMP'].dt.round('10T')
    # Optional: average values over 10-minute intervals (if desired)
    #df_combined_swr = df_combined_swr.groupby('TIMESTAMP').mean().reset_index()

    # Calculate solar position and clear-sky GHI
    timestamps = pd.DatetimeIndex(df_combined_swr['TIMESTAMP'])
    solar_position, clear_sky_ghi = calculate_solar_position_and_clear_sky(LATITUDE, LONGITUDE, timestamps)

    # Add calculated values to SWR DataFrame
    df_combined_swr['apparent_zenith'] = solar_position['apparent_zenith'].values
    df_combined_swr['clear_sky_ghi'] = clear_sky_ghi.values

    # Merge with LWP corrected data on TIMESTAMP
    merged_final_df = pd.merge(df_combined_swr, df_lwp_corrected, on='TIMESTAMP', how='inner')
    
    return df_combined_swr, merged_final_df



In [ ]:
def calculate_csi(df):
    """Calculate CSI and add it to the DataFrame."""
    df['CSI'] = df['SR15D1Dn_Irr'] / df['clear_sky_ghi']
    return df


In [ ]:
def plot_data(df):
    # Set font properties for presentation slides without bold text
    plt.rcParams.update({
    'font.size': 16,        # Increase the default font size
    'axes.titlesize': 20,   # Larger title size
    'axes.labelsize': 18,   # Larger label size
    'legend.fontsize': 16,  # Increase legend font size
    'xtick.labelsize': 14,  # Increase x-tick size
    'ytick.labelsize': 14,  # Increase y-tick size
        })

    """Plot the data for Shortwave Down Radiation vs. Solar Zenith Angle."""
    plt.figure(figsize=(10, 6))
    plt.plot(df['apparent_zenith'], df['SR15D1Dn_Irr'], 'ko', markersize=0.1, alpha=0.1, label='Measured SWR')
    plt.plot(df['apparent_zenith'], df['clear_sky_ghi'], 'ro', markersize=0.1, alpha=0.5, label='Clear-Sky GHI', zorder=5)
    
    plt.xlabel('Apparent Zenith Angle (degrees)',fontsize=16)
    plt.ylabel('$SW_{\downarrow}$ Radiation (W/m²)',fontsize=16)
    plt.title('$SW_{\downarrow}$ Radiation vs Solar Zenith Angle (March-June)',fontsize=18)
    #plt.legend()
    plt.grid(True, linestyle='--', linewidth=0.6, alpha=0.7)

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust rect to leave space for the title

    plt.show()


In [ ]:
def filter_clear_sky_days(df):
    """Filter the DataFrame for clear-sky conditions and low LWP."""
    # Filter for LWP_Corrected less than 50 and CSI values greater than 0.9
    filtered_df = df[(df['LWP_Corrected'] < 300)].reset_index(drop=True)#& (df['CSI'] > 0.9)].reset_index(drop=True)
    return filtered_df


In [ ]:
def plot_zenith_vs_sw_down(df):
    """Plot Apparent Zenith vs. SW Down Radiation with color map based on LWP values."""
    plt.figure(figsize=(10, 6))
    
      # Define bins for LWP_Corrected from 0 to 300 with 60 bins
    bins = np.linspace(0, 300, num=61)  # Create 60 bins from 0 to 300
    #cmap = plt.cm.viridis  # Use a continuous colormap like viridis for better clarity
    cmap = plt.cm.jet  # Use the jet colormap for coloring

    # Use `cut` to assign bin indices based on the LWP_Corrected values
    df['LWP_Binned'] = pd.cut(df['LWP_Corrected'], bins=bins, labels=False, include_lowest=True)

    # Create scatter plot with binned colors
    scatter = plt.scatter(
        df['apparent_zenith'], 
        df['SR15D1Dn_Irr'], 
        c=df['LWP_Corrected'],  # Use the actual LWP values for coloring
        cmap=cmap,
        s=1,                   
        alpha=0.5              
    )
    
    # Add a color bar with clear labeling
    cbar = plt.colorbar(scatter)
    cbar.set_label('LWP Corrected (g/m²)', fontsize=12)
    cbar.ax.tick_params(labelsize=10)  # Adjust tick size for readability
    
    
    # Set labels and title
    plt.xlabel('Apparent Zenith Angle (degrees)')
    plt.ylabel('$SW_{\downarrow}$ Radiation (W/m²)')
    plt.title('$SW_{\downarrow}$ Radiation vs Apparent Zenith (Colored by LWP)')
    
    plt.grid(True, linestyle='--', linewidth=0.6, alpha=0.7)

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust rect to leave space for the title

    plt.show()

In [ ]:
def main():
    global df_combined_swr, merged_final_df
    # Load and prepare the data only once
    df_combined_swr, merged_final_df = load_and_prepare_data()
    
    # Display the merged DataFrame without CSI
    print("\SWR combined with clear sky ghi:")
    print(df_combined_swr)
    print("\nMerged DataFrame without CSI:")
    print(merged_final_df.head())

if __name__ == "__main__":
    main()

In [ ]:
plot_data(df_combined_swr)  # Plot the data


In [ ]:
merged_final_df = calculate_csi(merged_final_df)  # Calculate CSI
print("\nMerged DataFrame with CSI:")
print(merged_final_df)


In [ ]:
# Filter for clear-sky days and low LWP
clear_sky_df = filter_clear_sky_days(merged_final_df)
print("\nMerged DataFrame filtered for LWP<300:")

print(clear_sky_df)
plot_zenith_vs_sw_down(clear_sky_df)


In [ ]:
plot_zenith_vs_sw_down(merged_final_df)


In [ ]:
def plot_binned_zenith_vs_sw_down_colored_by_lwp(df, lwp_bin_size=30, zenith_bin_size=3):
    """Plot SW Down Radiation vs. Apparent Zenith with color map based on binned LWP."""
    
    # Step 1: Bin LWP_Corrected in intervals of 5 and label the bins with their midpoint values
    lwp_min, lwp_max = df['LWP_Corrected'].min(), df['LWP_Corrected'].max()
    lwp_bins = np.arange(lwp_min, lwp_max + lwp_bin_size, lwp_bin_size)
    df['LWP_Binned'] = pd.cut(df['LWP_Corrected'], bins=lwp_bins, labels=lwp_bins[:-1] + lwp_bin_size / 2, include_lowest=True)
    
    # Step 2: Bin apparent_zenith and calculate mean SW Down Radiation and mean LWP for each zenith bin
    zenith_min, zenith_max = df['apparent_zenith'].min(), df['apparent_zenith'].max()
    zenith_bins = np.arange(zenith_min, zenith_max + zenith_bin_size, zenith_bin_size)
    df['Zenith_Binned'] = pd.cut(df['apparent_zenith'], bins=zenith_bins, labels=zenith_bins[:-1] + zenith_bin_size / 2, include_lowest=True)
    
    # Step 3: Group by Zenith_Binned and LWP_Binned and calculate the mean of SW Down Radiation and LWP
    binned_data = df.groupby(['Zenith_Binned', 'LWP_Binned']).agg({
        'SR15D1Dn_Irr': 'mean',
        'LWP_Corrected': 'mean'
    }).dropna().reset_index()

    # Plot the data
    plt.figure(figsize=(12, 7))
    
    # Use scatter plot to show SW Down Radiation with colors representing LWP levels
    scatter = plt.scatter(
        binned_data['Zenith_Binned'].astype(float),   # Convert to float for continuous axis
        binned_data['SR15D1Dn_Irr'], 
        c=binned_data['LWP_Corrected'],               # Color by mean LWP_Corrected
        cmap='jet',                                   # Jet colormap for a vibrant color range
        s=50,                                         # Size of scatter points
        edgecolor='k',                                # Outline to make points distinct
        alpha=0.7
    )
    
    # Add a color bar to indicate LWP values
    cbar = plt.colorbar(scatter)
    cbar.set_label('Mean LWP Corrected (g/m²)', fontsize=12)
    cbar.ax.tick_params(labelsize=10)
    
    # Set labels and title
    plt.xlabel('Binned Apparent Zenith Angle (degrees)')
    plt.ylabel('Mean SW Down Radiation (W/m²)')
    plt.title('Mean SW Down Radiation vs Binned Apparent Zenith (Colored by Mean LWP)')
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    
    plt.tight_layout()
    plt.show()



In [ ]:
plot_binned_zenith_vs_sw_down_colored_by_lwp(clear_sky_df)


In [ ]:
def plot_sw_down_parametrized_vs_binned_theta_with_lwp_colorbar(df_lwp, S0=1361, rho_l=1000, re=10,lwp_bin_size=10, theta_bin_size=3):
    """Plot SW Down Radiation vs. Binned Apparent Zenith with LWP as colorbar."""
    
    # Step 1: Calculate tau and A using the provided formulas
    # Calculate tau using the formula: tau = (3/2) * (LWP / (rho_l * re))
    df_lwp['tau'] = (3 / 2) * (df_lwp['LWP_Corrected'] * 1000 / (rho_l * re))  # Convert LWP to kg/m²
    # Calculate cloud transmission A using the formula: A = tau / (6.8 + tau)
    df_lwp['A'] = df_lwp['tau'] / (6.8 + df_lwp['tau'])  
    
    # Step 2: Extract solar zenith angle (theta) and calculate SW_toa (top of the atmosphere radiation)
    df_lwp['theta_0'] = df_lwp['apparent_zenith']  # Assuming solar_position['apparent_zenith'] is available
    df_lwp['SW_TOA'] = S0 * np.maximum(np.cos(np.radians(df_lwp['theta_0'])), 0)  # Calculate SW_toa
    
    # Step 3: Calculate SW Down Radiation (SW_dn = SW_toa * (1 - A))
    df_lwp['SW_dn_parameterized'] = df_lwp['SW_TOA'] * (1 - df_lwp['A'])  # Use the formula for SW_down
    
    # Step 4: Bin apparent_zenith (theta) and calculate the mean of SW_dn and LWP for each bin
    zenith_min, zenith_max = df_lwp['theta_0'].min(), df_lwp['theta_0'].max()
    zenith_bins = np.arange(zenith_min, zenith_max + theta_bin_size, theta_bin_size)
    df_lwp['Zenith_Binned'] = pd.cut(df_lwp['theta_0'], bins=zenith_bins, labels=zenith_bins[:-1] + theta_bin_size / 2, include_lowest=True)
     # Step 1: Bin LWP_Corrected in intervals of 5 and label the bins with their midpoint values
    lwp_min, lwp_max = df_lwp['LWP_Corrected'].min(), df_lwp['LWP_Corrected'].max()
    lwp_bins = np.arange(lwp_min, lwp_max + lwp_bin_size, lwp_bin_size)
    df_lwp['LWP_Binned'] = pd.cut(df_lwp['LWP_Corrected'], bins=lwp_bins, labels=lwp_bins[:-1] + lwp_bin_size / 2, include_lowest=True)
    
    # Step 5: Group by Zenith_Binned and calculate mean SW_dn and LWP for each zenith bin
    # Step 3: Group by Zenith_Binned and LWP_Binned and calculate the mean of SW Down Radiation and LWP
    binned_data = df_lwp.groupby(['Zenith_Binned', 'LWP_Binned']).agg({
        'SW_dn_parameterized': 'mean',
        'LWP_Corrected': 'mean'
    }).dropna().reset_index()
    
    # Step 6: Plot the data
    plt.figure(figsize=(12, 7))
    
    # Scatter plot with color map based on LWP_Corrected
    scatter = plt.scatter(
        binned_data['Zenith_Binned'].astype(float),  # Convert binned theta values to float for continuous axis
        binned_data['SW_dn_parameterized'],                       # SW Down Radiation on the y-axis
        c=binned_data['LWP_Corrected'],             # Color by LWP values
        cmap='jet',                             # Use 'viridis' for perceptually uniform colormap
        s=50,                                       # Size of scatter points
        edgecolor='k',                              # Black edge for distinct points
        alpha=0.7                                   # Transparency for overlapping points
    )
    
    # Add colorbar for LWP values
    cbar = plt.colorbar(scatter)
    cbar.set_label('Mean LWP Corrected (g/m²)', fontsize=12)
    cbar.ax.tick_params(labelsize=10)
    
    # Set labels and title
    plt.xlabel('Binned Apparent Zenith Angle (degrees)', fontsize=14)
    plt.ylabel('SW Down Radiation (W/m²)', fontsize=14)
    plt.title('SW Down Radiation vs Binned Apparent Zenith (Colored by Mean LWP)', fontsize=16)
    
    # Optional: Add grid and axis limits
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_sw_down_parametrized_vs_binned_theta_with_lwp_colorbar(clear_sky_df)

In [ ]:
def plot_binned_sw_down_vs_lwp_colored_by_zenith(df, lwp_bin_size=5, zenith_bin_size=3):
    """Plot SW Down Radiation vs. LWP with color map based on binned Zenith."""
    
    # Step 1: Bin LWP in intervals of `lwp_bin_size` and label the bins with midpoint values
    lwp_min, lwp_max = df['LWP_Corrected'].min(), df['LWP_Corrected'].max()
    lwp_bins = np.arange(lwp_min, lwp_max + lwp_bin_size, lwp_bin_size)
    df['LWP_Binned'] = pd.cut(df['LWP_Corrected'], bins=lwp_bins, labels=lwp_bins[:-1] + lwp_bin_size / 2, include_lowest=True)
    
    # Step 2: Bin apparent_zenith and calculate mean SW Down Radiation and mean Zenith for each zenith bin
    zenith_min, zenith_max = df['apparent_zenith'].min(), df['apparent_zenith'].max()
    zenith_bins = np.arange(zenith_min, zenith_max + zenith_bin_size, zenith_bin_size)
    df['Zenith_Binned'] = pd.cut(df['apparent_zenith'], bins=zenith_bins, labels=zenith_bins[:-1] + zenith_bin_size / 2, include_lowest=True)
    
    # Step 3: Group by LWP_Binned and Zenith_Binned to calculate the mean of SW Down Radiation and Zenith
    binned_data = df.groupby(['LWP_Binned', 'Zenith_Binned']).agg({
        'SR15D1Dn_Irr': 'mean',
        'apparent_zenith': 'mean'
    }).dropna().reset_index()

    # Plot the data
    plt.figure(figsize=(12, 7))
    
    # Scatter plot to show SW Down Radiation vs. LWP with colors representing Zenith
    scatter = plt.scatter(
        binned_data['LWP_Binned'].astype(float),      # Convert to float for continuous axis
        binned_data['SR15D1Dn_Irr'], 
        c=binned_data['apparent_zenith'],             # Color by mean apparent_zenith
        cmap='jet',                                   # Use jet colormap
        s=50,                                         # Size of scatter points
        edgecolor='k',                                # Outline for clarity
        alpha=0.7
    )
    
    # Add color bar for Zenith values
    cbar = plt.colorbar(scatter)
    cbar.set_label('Mean Apparent Zenith Angle (degrees)', fontsize=12)
    cbar.ax.tick_params(labelsize=10)
    
    # Set labels and title
    plt.xlabel('Binned LWP (g/m²)')
    plt.ylabel('Mean SW Down Radiation (W/m²)')
    plt.title('Mean SW Down Radiation vs Binned LWP (Colored by Mean Zenith Angle)')
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_binned_sw_down_vs_lwp_colored_by_zenith(clear_sky_df)

In [ ]:
# Edit before running!!! 
#Cabauw dataset path (enable only if you have this file)
# Example: cabauw_parquet_path = r"D:\Thesis\data\Cabauw\cabauw_merged_data_April_June_July.parquet"
cabauw_parquet_path = r"C:\path\to\your\Cabauw\cabauw_merged_data_April_June_July.parquet"

# Load Cabauw data
cabauw_df = pd.read_parquet(cabauw_parquet_path)
cabauw_df['TIMESTAMP'] = pd.to_datetime(cabauw_df['TIMESTAMP'], errors='coerce')

# Step 3: Call your existing function
timestamps = pd.DatetimeIndex(cabauw_df['TIMESTAMP'])
solar_position, clear_sky_ghi = calculate_solar_position_and_clear_sky(51.971, 4.927, timestamps)

# Step 4: Assign values back to DataFrame
cabauw_df['apparent_zenith'] = solar_position['apparent_zenith'].values
cabauw_df['clear_sky_ghi'] = clear_sky_ghi.values

print(cabauw_df)

In [ ]:
def plot_swd_vs_theta_lines_only_lwp_under_300(warmenhuizen_df, cabauw_df, lwp_bin_size=50, theta_bin_size=3):
    """
    Plot SW↓ vs apparent zenith angle for Warmenhuizen and Cabauw, grouped by LWP bins < 300 g/m².
    Warmenhuizen = solid lines, Cabauw = dashed lines. Uses colorbar for LWP bins.
    """
    # Filter for LWP < 300 g/m²
    warmenhuizen_df = warmenhuizen_df[warmenhuizen_df['LWP_Corrected'] < 300]
    cabauw_df = cabauw_df[cabauw_df['LWP (kg m-2)'] * 1000 < 300]

    # Bin LWP
    lwp_bins = np.arange(0, 301, lwp_bin_size)
    lwp_labels = [f"{int(b)}-{int(b + lwp_bin_size)}" for b in lwp_bins[:-1]]

    warmenhuizen_df['LWP_Binned'] = pd.cut(
        warmenhuizen_df['LWP_Corrected'],
        bins=lwp_bins,
        labels=lwp_labels,
        include_lowest=True
    )

    cabauw_df['LWP_Binned'] = pd.cut(
        cabauw_df['LWP (kg m-2)'] * 1000,
        bins=lwp_bins,
        labels=lwp_labels,
        include_lowest=True
    )

    # Bin zenith
    zenith_bins = np.arange(0, 95, theta_bin_size)
    warmenhuizen_df['Zenith_Binned'] = pd.cut(warmenhuizen_df['apparent_zenith'], bins=zenith_bins, labels=zenith_bins[:-1] + theta_bin_size / 2)
    cabauw_df['Zenith_Binned'] = pd.cut(cabauw_df['apparent_zenith'], bins=zenith_bins, labels=zenith_bins[:-1] + theta_bin_size / 2)

    # Plot setup
    fig, ax = plt.subplots(figsize=(12, 7))
    cmap = plt.cm.get_cmap('jet', len(lwp_labels))
    norm = mpl.colors.Normalize(vmin=0, vmax=len(lwp_labels) - 1)

    for i, lwp_label in enumerate(lwp_labels):
        color = cmap(i)

        # Warmenhuizen
        wh_bin = warmenhuizen_df[warmenhuizen_df['LWP_Binned'] == lwp_label]
        wh_grouped = wh_bin.groupby('Zenith_Binned', observed=True)['SR15D1Dn_Irr'].mean().dropna()
        if not wh_grouped.empty:
            ax.plot(wh_grouped.index.astype(float), wh_grouped.values,
                   color=color, linestyle='-', label=f'{lwp_label} (WH)')

        # Cabauw
        cb_bin = cabauw_df[cabauw_df['LWP_Binned'] == lwp_label]
        cb_grouped = cb_bin.groupby('Zenith_Binned', observed=True)['SWD'].mean().dropna()
        if not cb_grouped.empty:
             ax.plot(cb_grouped.index.astype(float), cb_grouped.values,
                    color=color, linestyle='--', label=f'{lwp_label} (Cabauw)')

    # Axis labels and grid
    ax.set_xlabel('Apparent Zenith Angle (°)', fontsize=14)
    ax.set_ylabel('SW↓ Radiation (W/m²)', fontsize=14)
    ax.set_title('SW↓ vs Apparent Zenith (LWP < 300 g/m²)', fontsize=16)
    #ax.grid(True, linestyle='--', alpha=0.7)

    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, ticks=np.arange(len(lwp_labels)))
    cbar.ax.set_yticklabels(lwp_labels)
    cbar.set_label('LWP Bin (g/m²)', fontsize=12)
    # Custom legend for line styles
    from matplotlib.lines import Line2D
    custom_lines = [
        Line2D([0], [0], color='black', linestyle='-', label='Warmenhuizen (solid)'),
        Line2D([0], [0], color='black', linestyle='--', label='Cabauw (dashed)')
    ]
    ax.legend(handles=custom_lines, loc='upper right', fontsize=10, title="Site")

    plt.tight_layout()
    plt.show()


In [ ]:
plot_swd_vs_theta_lines_only_lwp_under_300(clear_sky_df, cabauw_df)

In [ ]:
print(clear_sky_df)

In [ ]:
print(cabauw_df)